## LR (ligand and Receptor) networks
This notebook analyzes all L-R dataframes obtained from NATMI. 
Checks edges' weight distribution, tests different thresholds for filtering and saves filtered and unfiltered LR dataframes (as a dictionary of `LRinfos` objects). It also generates and saves filtered and unfiltered networkx objects dictionaries. 
-------
Author : johaGL

In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
#import holoviews
import pygraphviz
import networkx as nx
import pandas as pd
import matplotlib.patches as mpatches
from scipy import stats
import pickle
import igraph as ig

In [ ]:
import pylab as pyl

In [ ]:
class LRinfos:  
    """
    class to handle  Natmi dataframe results
    one object by result ! 
    to create object, predf is needed (the Edges opened with pandas csv)
    use 'frame' attribute to get dataframe suitable for graph conversion
    """
    def __init__(self, age, day, predf):
        self.age = age
        self.day = day
        self.predf  = predf
        self.makeunique_symbo_cellty()
        self.about = f"object age {age}, day {day}, use 'frame' attribute for more!"
        
    def makeunique_symbo_cellty(self):
        otab = self.predf
        otab['uniq_Ligand_symbol'] = otab['Ligand symbol'] + '_' + otab['Sending cluster']
        otab['uniq_Receptor_symbol'] = otab['Receptor symbol'] + '_' + otab['Target cluster']
        self.frame = otab  # this adds attribute 'frame'     
        
    def filterZero(self):  
        tmp = self.frame.loc[self.frame['Edge average expression derived specificity'] > 0]
        self.frame = tmp  # yield only non zero edges dataframe
    
    def filterOnEdgeslog10(self, cutoff):
        if min(self.frame['Edge average expression derived specificity']) < 0:
            self.filterZero()
        self.frame['log10_edge_sp'] = np.log10(np.array(self.frame['Edge average expression derived specificity']))
        tf = self.frame.loc[ self.frame['log10_edge_sp'] >= cutoff ]
        self.filtered = tf
        
           

In [ ]:
ages = [ 'Young', 'Old']
days = ['D0', 'D2', 'D4', 'D7']     

<div class="alert alert-info">
  <strong>Network files : </strong>
     These tab delimited dataframe files are named identically, 
     So, what distinguishes Networks is folder location, folder location has age and day
    see variable: 
      <p style="font-family:'Lucida Console', monospace">deffilename</p>
</div> 

`deffilename`

In [ ]:
"""
Network files are named identically (deffilename) , BUT : 
what distinguishes Networks is folder location, folder location has age and day !!!
"""
indatadir = "~/BulkAnalysis_plusNetwork/NatmiData/natmiOut_TPM/"
print(os.getcwd())
deffilename = 'Network_exp_0_spe_0_det_0.6_top_0_signal_lrc2p_weight_mean/'

# example opening a file: 
old_D7 = pd.read_csv(f'{indatadir}OldD7/{deffilename}Edges.csv',sep=",", header=0) 
old_D7.head(1)

In [ ]:
"""
all LRinfos objects are stocked in dictionary
"""
lr = {}      
for i in ages:
    lr[i] = {}
    for j in days:   
        predf = pd.read_csv(f'{indatadir}{i}{j}/{deffilename}Edges.csv',sep=",", header=0)  
        myob = LRinfos(i, j, predf)
        lr[i][j] = myob      

In [ ]:
print(lr['Young']['D2'].about)

In [ ]:
lr['Young']['D2'].frame.head(1)

In [ ]:
max(lr['Old']['D0'].frame['Edge average expression derived specificity'])

In [ ]:
# by default , Natmi yields edges exhibiting specificities > 0
# verify : 
for age in ages:
    for day in days:
        if (min(lr[age][day].frame['Edge average expression derived specificity']) > 0):
            print(f'{age}+{day} specificities edges are all over zero')
        else : 
            print(f'{age}+{day} HAS A MIN VALUE EQUAL TO ZERO, FILTER OUT using obj.filterZero()')        

In [ ]:
""" plotting edge specificities """
histocols = ['orange', 'dimgray']
fig, axs = plt.subplots(2, 4, figsize=(9,4))
for rawi in range(2):
    AGE = ages[rawi]
    for coli in range(4):
        DAY = days[coli]
        tmpvec = lr[AGE][DAY].frame['Edge average expression derived specificity']        
        axs[rawi, coli].hist(tmpvec,
                            color=histocols[rawi], bins=40)
        axs[rawi, coli].set_title(f'{ages[rawi]} {days[coli]}')
        

for ax in axs.flat:
    ax.set(xlabel='values', ylabel='n')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

fig.suptitle(f'Edge specificities, given values', fontsize=16)

**This is very similar to TAu indexes, being 1 most specific, and 0 housekeeping !**  . Most have weak specificities, do preservative approach, by using log10 to filter. First we must explore log transformed values

In [ ]:
""" plotting edge specificities, log """

histocols = ['orange', 'dimgray']
fig, axs = plt.subplots(2, 4, figsize=(10,4))
for rawi in range(2):
    AGE = ages[rawi]
    for coli in range(4):
        DAY = days[coli]
        tmpvec = lr[AGE][DAY].frame['Edge average expression derived specificity']        
        axs[rawi, coli].hist(np.log10(np.array(tmpvec)),
                            color=histocols[rawi], bins=40)
        axs[rawi, coli].set_title(f'{ages[rawi]} {days[coli]}')        

for ax in axs.flat:
    ax.set(xlabel='log10 values', ylabel='n')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

fig.suptitle(f'Edge specificities, log10 (x)', fontsize=16)

The closer to 1 (0 in log10 value) the more the edge is predicted to be 'specific'
We see that most of edges specificities are concentrated between 0.1 and 0.001 (-1 and -3 in log10 value). 
These plots allowed us to know about data distribution, but we need
to scale them to  values suitable for making graph style visualizations.

### filter , using function from my 'LRinfos' class : 

The function 'filterOnEdgeslog10' is defined inside the class (above), cutoff is customizable

In [ ]:
## filter and check edges
for age in ages:
    for day in days:
        lr[age][day].filterOnEdgeslog10(-0.5)   # using -0.5 as log10 cutoff

In [ ]:
lr['Young']['D4'].filtered['log10_edge_sp']

In [ ]:
## plotting POST filter
""" plotting POST filter edge specificities """
histocols = ['orange', 'dimgray']
fig, axs = plt.subplots(2, 4, figsize=(12,5))
for rawi in range(2):
    AGE = ages[rawi]
    for coli in range(4):
        DAY = days[coli]
        tmpvec = lr[AGE][DAY].filtered['Edge average expression derived specificity']     
        #tmpvec = lr[AGE][DAY].filtered['log10_edge_sp']
        axs[rawi, coli].hist(np.array(tmpvec),
                            color=histocols[rawi], bins=40)
        axs[rawi, coli].set_title(f'{ages[rawi]} {days[coli]}')
        

for ax in axs.flat:
    ax.set(xlabel='values', ylabel='n')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

fig.suptitle(f'Edge specificities, POST filter ', fontsize=16)

In [ ]:
for age in ages:
    for day in days:
        print((age,day))
        print(lr[age][day].filtered.shape)

That was very stringent (-0.5), let's set another cutoff

In [ ]:
for age in ages:
    for day in days:
        lr[age][day].filterOnEdgeslog10(-1) 

### Compare filtered and unfiltered number of edges:
Making a simple table

In [ ]:
tabularmanual = {'notfiltered' : [], 'filtered' : []}
manualindexes = []
for i in ages:
    for j in days:
        tabularmanual['notfiltered'].append(lr[i][j].frame.shape[0])
        tabularmanual['filtered'].append(lr[i][j].filtered.shape[0])
        manualindexes.append(f'{i}_{j}')
#    = {'notfiltered' : [1,2,3], 'filtered' : [4,5,6]}

In [ ]:
lr['Old']['D0'].filtered.shape[0]

In [ ]:
print("compare unfiltered vs filtered NUMBER OF EDGES, as filter done on edges specs")
pd.DataFrame(tabularmanual, index = manualindexes)

In [ ]:
""" plotting POST filter edge specificities """
histocols = ['orange', 'dimgray']
fig, axs = plt.subplots(2, 4, figsize=(10,5))
for rawi in range(2):
    AGE = ages[rawi]
    for coli in range(4):
        DAY = days[coli]
        tmpvec = lr[AGE][DAY].filtered['Edge average expression derived specificity']     
        #tmpvec = lr[AGE][DAY].filtered['log10_edge_sp']
        axs[rawi, coli].hist(np.array(tmpvec),
                            color=histocols[rawi], bins=40)
        axs[rawi, coli].set_title(f'{ages[rawi]} {days[coli]}')
        

for ax in axs.flat:
    ax.set(xlabel=' values', ylabel='n')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

fig.suptitle(f'Edge specificities, POST filter ', fontsize=16)

## New function to get Networkx object from dataframe
#### and give it colors as universally defined, from specialized color-blindness friendly palette (defined from R scripts)

In [ ]:
def getCustomGraph(adf, celltycolors):
    """Inputs :  adf (a dataframe) , and
                 colors in the form of a dictionary.
    Output:  Networkx object"""
    G = nx.DiGraph()
    for index,row in adf.iterrows():
        nodefrom = row['uniq_Ligand_symbol']
        nodeto = row['uniq_Receptor_symbol']
        celltypefrom = row['Sending cluster']
        celltypeto = row['Target cluster']
        G.add_node(nodefrom,
                    nodetype = 'sender',
                    celltype = celltypefrom,
                    genesym = row['Ligand symbol'],
                    color = celltycolors[celltypefrom],
                    specificity = row['Ligand derived specificity of average expression value'])
        G.add_node(nodeto,
                    nodetype = 'receiver',
                    celltype = celltypeto, 
                    genesym = row['Receptor symbol'],
                    color = celltycolors[celltypeto],
                    specificity = row['Receptor derived specificity of average expression value'])
        G.add_edge(nodefrom,nodeto, origtype = nodefrom,
                    ecolor = celltycolors[celltypefrom],
                    weight = row['Edge average expression derived specificity']) 
    return G

In [ ]:
celltycolors = {
  "ECs" : "#10b387ff",
  "FAPs" : "#3d85c6ff",
    "MuSCs" : "#b171f1ff",
  "Neutrophils" : "#f0e442ff",
  "Inflammatory-Mac" : "#ff9900ff",
  "Resolving-Mac" : "#cc0000ff"
}

In [ ]:
## creating new dictionary to stock Networkx objects
dxfiltered = {}
dxunfiltered = {}

In [ ]:
## calling function iteratively while populating dictionary
for age in ages:
    dxfiltered[age] = {}
    dxunfiltered[age] = {}
    for day in days:
        tmpGfi = getCustomGraph(lr[age][day].filtered, celltycolors)
        tmpGuf = getCustomGraph(lr[age][day].frame, celltycolors)
        dxfiltered[age][day] = tmpGfi
        dxunfiltered[age][day] = tmpGuf
        

In [ ]:
len(dxfiltered['Old']['D4'].nodes())

In [ ]:
len(dxunfiltered['Old']['D4'].nodes())

## Saving pdf figures and  dictionaries (lr, dxfiltered and dxunfiltered):

 * Saving pdf figures whole networks

In [ ]:
for age in ages:
    for day in days:        
        G = dxfiltered[age][day]
        ccc = [ i for i in nx.get_node_attributes(G, 'color').values()]
        eee = [ i for i in nx.get_edge_attributes(G, 'ecolor').values()]## do plot
        pos = nx.spring_layout(G)

        fig = plt.figure(figsize=(30,25),dpi=150)
        color_patches = []
        for key, value in celltycolors.items():
            tmppatch = mpatches.Patch(color=value, label=key)
            color_patches.append(tmppatch)

        nodes = nx.draw_networkx_nodes(G,
                                      pos,
                                      node_color = ccc, edgecolors = 'lightgray', 
                                      alpha=0.7)
        edges = nx.draw_networkx_edges(G,
                                      pos,
                                      edge_color = eee, alpha=0.7)
        ax = plt.gca()
        ax.set_axis_off()
        fig.tight_layout()
        plt.legend(handles=color_patches, prop={'size': 20})

        fig.suptitle(f'{age}, {day}. n edges:{len(G.edges())}', fontsize=22)
        fig.savefig(f'postFilterNet_{age}{day}.pdf', dpi=150)

* obtaining huge size legend for graphical editing purposes

In [ ]:
color_patches = []
for key, value in celltycolors.items():
    tmppatch = mpatches.Patch(color=value, label=key)
    color_patches.append(tmppatch)
    
onlylegend = plt.figure()
plt.legend(handles=color_patches, prop={'size': 20})
onlylegend.suptitle('Only Legend for plotting purposes whenever needed')
onlylegend.savefig('onlyLegend.pdf')

 * Saving files
    - lr : contains objects of the class 'LRinfos' which I created to stock dataframes
            - `.frame` attribute contains unfiltered dataframe
            - `.filtered` attribute contains filtered dataframe
    - dxfiltered and dxunfiltered : contain networkx objects

    saving  dictionaries (lr  and filtered and unfiltered graphs dictionaries) into different files

In [ ]:
pickle.dump(lr, open( "graphobjs/dictio_lr.p", "wb" ))

In [ ]:
pickle.dump(dxfiltered, open( "graphobjs/dictio_dx_filtered.p", "wb" ))

In [ ]:
pickle.dump(dxunfiltered, open( "graphobjs/dictio_dx_unfi.p", "wb" ))

In [ ]:
# to open just: lr = pickle.load( open( "graphobjs/dictio_lr.p", "rb" ) )

In [ ]:
plt.close('all')

## Save interoperable igraph objects

For some operations, networkx is more robust than igraph (for example, for transformating into undirected graph, or checking number of connected components for directed graph). However, as we are going to use output to import in a Shiny application, I have decided to use igraph objects for interoperability purposes.

In [ ]:
dxfiltered = pickle.load( open('graphobjs/dictio_dx_filtered.p', 'rb') )
dxunfiltered = pickle.load( open('graphobjs/dictio_dx_unfi.p', 'rb') )

In [ ]:
ages = ["Young", "Old"]
days = ["D0", "D2", "D4", "D7"]
for age in ages:
    print(age)
    for day in days:
        tmp_igraph_filt = ig.Graph.from_networkx(dxfiltered[age][day])
        tmp_igraph_unfi = ig.Graph.from_networkx(dxunfiltered[age][day])
        tmp_igraph_filt.write_graphml(f'graphobjs/{age}_{day}_igraph_filt.ml')
        tmp_igraph_unfi.write_graphml(f'graphobjs/{age}_{day}_igraph_unfi.ml')

In [ ]:
# Same for data with normalisation deseq2

"""
Network files are named identically (deffilename) , BUT : 
what distinguishes Networks is folder location, folder location has age and day !!!
"""
indatadir = "~/BulkAnalysis_plusNetwork/NatmiData/natmiOut_CountNormalised/"
print(os.getcwd())
deffilename = 'Network_exp_0_spe_0_det_0.6_top_0_signal_lrc2p_weight_mean/'

# example opening a file: 
old_D7 = pd.read_csv(f'{indatadir}OldD7/{deffilename}Edges.csv',sep=",", header=0) 
old_D7.head(1)


"""
all LRinfos objects are stocked in dictionary
"""
lr = {}      
for i in ages:
    lr[i] = {}
    for j in days:   
        predf = pd.read_csv(f'{indatadir}{i}{j}/{deffilename}Edges.csv',sep=",", header=0)  
        myob = LRinfos(i, j, predf)
        lr[i][j] = myob  
        
print(lr['Young']['D2'].about)
lr['Young']['D2'].frame.head(1)

for age in ages:
    for day in days:
        lr[age][day].filterOnEdgeslog10(-0.5)   # using -0.5 as log10 cutoff
lr['Young']['D4'].filtered['log10_edge_sp']

for age in ages:
    for day in days:
        print((age,day))
        print(lr[age][day].filtered.shape)
for age in ages:
    for day in days:
        lr[age][day].filterOnEdgeslog10(-1) 
        
tabularmanual = {'notfiltered' : [], 'filtered' : []}
manualindexes = []
for i in ages:
    for j in days:
        tabularmanual['notfiltered'].append(lr[i][j].frame.shape[0])
        tabularmanual['filtered'].append(lr[i][j].filtered.shape[0])
        manualindexes.append(f'{i}_{j}')
#    = {'notfiltered' : [1,2,3], 'filtered' : [4,5,6]}
print("compare unfiltered vs filtered NUMBER OF EDGES, as filter done on edges specs")
pd.DataFrame(tabularmanual, index = manualindexes)

In [ ]:
dxfiltered = {}
dxunfiltered = {}
for age in ages:
    dxfiltered[age] = {}
    dxunfiltered[age] = {}
    for day in days:
        tmpGfi = getCustomGraph(lr[age][day].filtered, celltycolors)
        tmpGuf = getCustomGraph(lr[age][day].frame, celltycolors)
        dxfiltered[age][day] = tmpGfi
        dxunfiltered[age][day] = tmpGuf

In [ ]:
for age in ages:
    for day in days:        
        G = dxfiltered[age][day]
        ccc = [ i for i in nx.get_node_attributes(G, 'color').values()]
        eee = [ i for i in nx.get_edge_attributes(G, 'ecolor').values()]## do plot
        pos = nx.spring_layout(G)

        fig = plt.figure(figsize=(30,25),dpi=150)
        color_patches = []
        for key, value in celltycolors.items():
            tmppatch = mpatches.Patch(color=value, label=key)
            color_patches.append(tmppatch)

        nodes = nx.draw_networkx_nodes(G,
                                      pos,
                                      node_color = ccc, edgecolors = 'lightgray', 
                                      alpha=0.7)
        edges = nx.draw_networkx_edges(G,
                                      pos,
                                      edge_color = eee, alpha=0.7)
        ax = plt.gca()
        ax.set_axis_off()
        fig.tight_layout()
        plt.legend(handles=color_patches, prop={'size': 20})

        fig.suptitle(f'{age}, {day}. n edges:{len(G.edges())}', fontsize=22)
        fig.savefig(f'postFilterNet_{age}{day}_counNormalided.pdf', dpi=150)

In [ ]:
color_patches = []
for key, value in celltycolors.items():
    tmppatch = mpatches.Patch(color=value, label=key)
    color_patches.append(tmppatch)
    
onlylegend = plt.figure()
plt.legend(handles=color_patches, prop={'size': 20})
onlylegend.suptitle('Only Legend for plotting purposes whenever needed')
onlylegend.savefig('onlyLegend.pdf')

In [ ]:
pickle.dump(lr, open( "graphobjs/dictio_lr_CN.p", "wb" ))
pickle.dump(dxfiltered, open( "graphobjs/dictio_dx_filtered_CN.p", "wb" ))
pickle.dump(dxunfiltered, open( "graphobjs/dictio_dx_unfi_CN.p", "wb" ))
plt.close('all')

In [ ]:
dxfiltered = pickle.load( open('graphobjs/dictio_dx_filtered_CN.p', 'rb') )
dxunfiltered = pickle.load( open('graphobjs/dictio_dx_unfi_CN.p', 'rb') )
ages = ["Young", "Old"]
days = ["D0", "D2", "D4", "D7"]
for age in ages:
    print(age)
    for day in days:
        tmp_igraph_filt = ig.Graph.from_networkx(dxfiltered[age][day])
        tmp_igraph_unfi = ig.Graph.from_networkx(dxunfiltered[age][day])
        tmp_igraph_filt.write_graphml(f'graphobjs/{age}_{day}_igraph_filt_CN.ml')
        tmp_igraph_unfi.write_graphml(f'graphobjs/{age}_{day}_igraph_unfi_CN.ml')